In [1]:
from scipy import stats
import numpy as np

In [2]:
import re
import spacy
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 
stemmer = SnowballStemmer("english")
nlp = spacy.load('en')
MAX_CHARS = 20000
def tokenizer(comment):
    comment = comment.lower()
    comment = re.sub(r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’;#]`", " ", str(comment))
    comment = re.sub(r"[ ]+", " ", comment)
    comment = re.sub(r"\,+", ",", comment)
    if (len(comment) > MAX_CHARS):
        comment = comment[:MAX_CHARS]
    return [x.text for x in nlp.tokenizer(comment) if x.text != " "]

In [22]:
import csv
import collections
import string
from stop_words import get_stop_words

stop_words = get_stop_words('en')

word_dict = collections.defaultdict(list)
polarities = []

with open('data/csv/train/bluetooth.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    next(csvReader)
    for row in csvReader:
        sentiment = row[1]
        if sentiment == 'negative':
            polarity = -1
        else:
            polarity = 1
        polarities.append(polarity)
        texts = row[2]
        segs = tokenizer(texts)
        for seg in segs:
            if seg == 'edu_break' or seg == '-lrb-' or seg == '-rrb-' or seg in string.punctuation or seg.isdigit() or seg in stop_words or seg[0] == "'":
                continue
            word_dict[seg.lower()].append(polarity)
            

In [51]:
count = 0
with open('data/csv/train/bluetooth.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    next(csvReader)
    for row in csvReader:
        count+=1
print(count)

80148


In [31]:
mean_polarity = stats.tmean(polarities)

In [24]:
counts = []
for word in word_dict:
    counts.append(len(word_dict[word]))
counts = np.array(counts)
hist,bins=np.histogram(counts,bins=np.linspace(0,600000,600))

In [25]:
stats.tmean(counts)

82.04133296190543

In [26]:
stats.tstd(counts)

954.3157123117777

In [27]:
stats.scoreatpercentile(counts, 99)

1434.6500000000015

In [28]:
print(hist)

[56661   329   129    82    50    40    30    16    16    13    11     7
     3     5     3     8     0     4     3     3     1     1     3     1
     1     1     1     0     0     1     1     0     0     1     0     0
     0     1     1     0     2     0     0     1     1     0     0     1
     0     0     1     1     0     0     0     0     0     0     0     0
     0     1     0     0     0     1     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [29]:
print(bins)

[     0.           1001.66944908   2003.33889816   3005.00834725
   4006.67779633   5008.34724541   6010.01669449   7011.68614357
   8013.35559265   9015.02504174  10016.69449082  11018.3639399
  12020.03338898  13021.70283806  14023.37228715  15025.04173623
  16026.71118531  17028.38063439  18030.05008347  19031.71953255
  20033.38898164  21035.05843072  22036.7278798   23038.39732888
  24040.06677796  25041.73622705  26043.40567613  27045.07512521
  28046.74457429  29048.41402337  30050.08347245  31051.75292154
  32053.42237062  33055.0918197   34056.76126878  35058.43071786
  36060.10016694  37061.76961603  38063.43906511  39065.10851419
  40066.77796327  41068.44741235  42070.11686144  43071.78631052
  44073.4557596   45075.12520868  46076.79465776  47078.46410684
  48080.13355593  49081.80300501  50083.47245409  51085.14190317
  52086.81135225  53088.48080134  54090.15025042  55091.8196995
  56093.48914858  57095.15859766  58096.82804674  59098.49749583
  60100.16694491  61101.836

In [30]:
np.random.seed(1)

In [32]:
stats.ttest_1samp(word_dict['annoying'],mean_polarity)

Ttest_1sampResult(statistic=-24.805672332282107, pvalue=6.860970192622316e-124)

In [33]:
stats.ttest_1samp(word_dict['awesome'],mean_polarity)

Ttest_1sampResult(statistic=23.927934390123713, pvalue=2.072652578365373e-115)

In [39]:
#select seeds


pool = []
for word in word_dict:
    if len(word_dict[word]) < 500:
        continue
    ttest = stats.ttest_1samp(word_dict[word],mean_polarity)
    pool.append((word, ttest.statistic, ttest.pvalue))

In [40]:
words, tstats, pvals = zip(*pool)

In [41]:
pvals

(0.9477408267031945,
 3.4577678599026365e-28,
 0.0993523805703532,
 1.320429845977233e-07,
 4.5896723621160946e-70,
 1.624656972843774e-39,
 0.6200716036874552,
 0.0035476764596688304,
 0.0002693892117537446,
 1.6984675057236383e-300,
 8.694130614480968e-24,
 0.0018421831793758404,
 3.727472551369491e-42,
 7.039003979561349e-36,
 1.3059212269120026e-97,
 2.615370179216981e-19,
 7.764136709564804e-93,
 4.8768021569173925e-42,
 3.688060493105582e-222,
 0.017739094644693188,
 1.0485323076774317e-06,
 0.00037577203412846494,
 0.6902449930334374,
 1.5481840542752828e-16,
 3.5204320014739265e-302,
 0.003667868721030058,
 0.00019141114016363387,
 0.7086072343213732,
 0.037557216946880044,
 2.9496126366964754e-100,
 4.50558933621835e-06,
 1.433346976136325e-05,
 4.215347786629593e-15,
 0.035979902658844064,
 1.9072473309312724e-43,
 4.634395502490627e-17,
 1.025755566726004e-18,
 1.8825550790947356e-10,
 1.9464605865099473e-79,
 2.6981850201040863e-47,
 3.2350548484009454e-08,
 1.7328328518358

In [44]:
seeds = []
pos_count = 0
neg_count = 0
for word, tstat, pval in pool:
    if pval < 1e-100:
        seeds.append((word, tstat))
        if tstat > 0:
            pos_count += 1
        else:
            neg_count += 1

In [45]:
sorted_seeds = sorted(seeds, key=lambda x: abs(x[1]), reverse=True)

In [49]:
sorted_seeds

[('return', -55.050873158318645),
 ('love', 48.76846025907272),
 ('highly', 45.60070078470203),
 ('great', 43.54118419841544),
 ('refund', -43.191334635505456),
 ('returning', -40.712187529976404),
 ('however', -38.79117265182937),
 ('useless', -38.2025341662007),
 ("n't", -37.24576552367123),
 ('returned', -36.851681757965856),
 ('poor', -36.32192924216045),
 ('stars', -36.307806180363826),
 ('best', 34.53701167177729),
 ('bad', -34.41852610785666),
 ('unfortunately', -34.34469600871973),
 ('stopped', -33.44950316919172),
 ('back', -32.442452858901454),
 ('waste', -31.249966736635812),
 ('ok', -31.151909858262975),
 ('reason', -29.376322473677632),
 ('warranty', -29.34800739702489),
 ('amazing', 28.968987489891674),
 ('star', -28.519161553993364),
 ('defective', -26.763491221958127),
 ('terrible', -26.13653080617382),
 ('excellent', 26.02330657525055),
 ('easy', 25.999336939952965),
 ('plastic', -25.982130528770217),
 ('loves', 25.665122943497607),
 ('worse', -25.616955350032544),
 ('

In [46]:
num_ = min(pos_count, neg_count)
pos_ = 0
neg_ = 0
res_pos = []
res_neg = []
for seed, pol in sorted_seeds:
    if pol > 0 and pos_ < num_:
        res_pos.append(seed)
        pos_ += 1
    if pol < 0 and neg_ < num_:
        res_neg.append(seed)
        neg_ += 1
    

In [47]:
res_pos

['love',
 'highly',
 'great',
 'best',
 'amazing',
 'excellent',
 'easy',
 'loves',
 'recommend',
 'awesome',
 'clear']

In [48]:
res_neg

['return',
 'refund',
 'returning',
 'however',
 'useless',
 "n't",
 'returned',
 'poor',
 'stars',
 'bad',
 'unfortunately']